In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
import keras

In [ ]:
data = []
labels = []
classes = 43
cur_path = os.getcwd()
print(os.path.join(cur_path,'Traffic','train'))

In [ ]:
temp=0
for i in range(classes):
    path = os.path.join('../input/traffic-sign/traffic/Train',str(i))
    
    images = os.listdir(path)
    
    for a in images:
        
        try:
            image = Image.open(path + '/'+ a)
            
            temp+=1
            
            image = image.resize((30,30))
            
            image = np.array(image)
            data.append(image)
            labels.append(i)
           
        except:
            print("Error loading image")
print(temp)

In [ ]:
#Converting lists into numpy arrays
data = np.array(data)
labels = np.array(labels)

print(data.shape, labels.shape)

In [ ]:
#Splitting training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
#Converting the labels into one hot encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu', padding='same', input_shape=X_train.shape[1:]))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Conv2D(filters=128, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
#Compilation of the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 40
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))
model.save("my_model.h5")


In [ ]:
from tabulate import tabulate

table=[['Epoch','Accuracy','Loss','Val Accuracy','Val Loss']]
for i in range(epochs-1):
    temp=[str(i)]
    temp.append(str(float("{0:.4f}".format(history.history['accuracy'][i]))*100))
    temp.append(str(float("{0:.4f}".format(history.history['loss'][i]))))
    temp.append(str(float("{0:.4f}".format(history.history['val_accuracy'][i]))*100))
    temp.append(str(float("{0:.4f}".format(history.history['val_loss'][i]))))
    table.append(temp)
print(tabulate(table, headers='firstrow', tablefmt='fancy_grid'))

In [ ]:
table2=[['Max Accuracy','Min Loss','Max Val Accuracy','Min Val loss']]

temp=[str(max(history.history['accuracy'])*100),str(min(history.history['loss'])),str(max(history.history['val_accuracy'])*100),str(min(history.history['accuracy']))]
table2.append(temp)

print(tabulate(table2, headers='firstrow', tablefmt='fancy_grid'))

In [ ]:
#plotting graphs for accuracy 
fig, axs = plt.subplots(2, 2,figsize=(14,10))
axs[0, 0].plot(history.history['accuracy'], label='training accuracy')
axs[0, 0].plot(history.history['val_accuracy'], label='val accuracy')
axs[0, 0].set_title('Accuracy vs Val Accuracy')
axs[0, 0].set_xlabel('epochs')
axs[0, 0].set_ylabel('accuracy')



axs[1, 0].plot(history.history['loss'], label='training loss')
axs[1, 0].plot(history.history['val_loss'], label='val loss')
axs[1, 0].set_title('Loss vs Val Loss')
axs[1, 0].set_xlabel('epochs')
axs[1, 0].set_ylabel('loss')

axs[0, 1].plot(history.history['loss'], label='training loss')
axs[0, 1].plot(history.history['accuracy'], label='trainning accuracy')
axs[0, 1].set_title('Loss vs Accuracy')
axs[0, 1].set_xlabel('epochs')
axs[0, 1].set_ylabel('loss and accuracy')



axs[1, 1].plot(history.history['val_accuracy'], label='val accuracy')
axs[1, 1].plot(history.history['val_loss'], label='val loss')
axs[1, 1].set_title('Val Loss vs Val Accuracy')
axs[1, 1].set_xlabel('epochs')
axs[1, 1].set_ylabel('loss and accuracy')

fig.tight_layout()